In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


## Define the Class

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


## Model Layers

In [7]:
input_img = torch.rand(3, 28, 28)
print(input_img.shape)

torch.Size([3, 28, 28])


### nn.Flatten

In [8]:
flatten = nn.Flatten()
flat_img = flatten(input_img)
print(flat_img.size())

torch.Size([3, 784])


### nn.Linear

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_img)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1424,  0.5811,  0.4463, -0.2087, -0.1117, -0.1055,  0.1156, -0.0686,
          0.0010,  0.1516, -0.4738, -0.5436,  0.3060, -0.2149, -0.1485,  0.0808,
         -0.3877, -0.6122, -0.5552,  0.1332],
        [ 0.3636,  0.3867,  0.5270, -0.3928,  0.0055, -0.0426,  0.1887, -0.0724,
         -0.1871, -0.0150, -0.6023, -0.5169,  0.3618, -0.8116, -0.1562, -0.2563,
         -0.2968, -0.2811, -0.7538,  0.0688],
        [-0.0444,  0.6454,  0.6043, -0.5594,  0.2712,  0.0565,  0.1708, -0.1759,
          0.1834, -0.3160, -0.7956, -0.5366,  0.6837, -0.4280,  0.2117,  0.1508,
         -0.3299, -0.5855, -0.4356, -0.2201]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1424, 0.5811, 0.4463, 0.0000, 0.0000, 0.0000, 0.1156, 0.0000, 0.0010,
         0.1516, 0.0000, 0.0000, 0.3060, 0.0000, 0.0000, 0.0808, 0.0000, 0.0000,
         0.0000, 0.1332],
        [0.3636, 0.3867, 0.5270, 0.0000, 0.0055, 0.0000, 0.1887, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.3618, 0.0000, 0.00

### nn.Sequential

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_iamge = torch.rand(3, 28, 28)
logits = seq_modules(input_iamge)
print(f'logits shape: {logits.size()}')

logits shape: torch.Size([3, 10])


### nn.Softmax

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab.shape)

torch.Size([3, 10])


## Model Parameters

In [13]:
print(f"Model structure:\n {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:
 NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0227, -0.0050,  0.0127,  ...,  0.0348, -0.0121,  0.0065],
        [ 0.0316,  0.0167, -0.0131,  ...,  0.0241,  0.0205,  0.0224]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0083, -0.0056], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0439,  0.0161,  0.0302,  ..., -0.0214,  0.0083,  0.0127],
        [-0.0256,  0.0267, -0.0392,  ...,  0.0266,  0.0067,  0.0418]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias |